In [ ]:
#General Libraries:
import pandas as pd
import numpy as np
import os

#Web Scrapping Libraries:
from urllib.request import urlopen
from bs4 import BeautifulSoup

#Error Handling 
import sys
import traceback
import datetime

#Seeting up folder path:
os.chdir('C:/Users/mohdn/OneDrive/Desktop/Nadeem/DataSets/Reviews_Tool/')

In [ ]:
#Creating the dataframe to store the scrapped values

dataframe_comp=pd.DataFrame()
log = pd.DataFrame()

#Scraping Code:
#e='Exception message : HTTP Error 503: Service Unavailable'
while input('Do you want to scrap : ') == 'Yes':
    i=1
    e='Exception message : HTTP Error 503: Service Unavailable'
    
    country = input('''Choose the country from the list :
                    1 - US,
                    2 - UK,
                    3 - India,
                    4 - Canada,
                    5 - Australia
                    ''')

    product = input('Enter the product id :')
    page_count = input('''Enter the amount of pages to scrap
                        Minimum - 1
                        Maximum - 500
                        ''')
    
    while (i <=int(page_count)  and e==('Exception message : HTTP Error 503: Service Unavailable')):
        try:

            currentDT = datetime.datetime.now()
            ltime = str(currentDT.strftime("%Y-%m-%d %H:%M:%S"))
            ltimeandi = pd.DataFrame({'Iteration':i, 'Time':ltime},index=[0])
            log = log.append(ltimeandi,ignore_index=True)
            print(i)
            
            if country == '1':
                html = 'https://www.amazon.com/Echo-Dot-3rd-Gen-speaker/product-reviews/{}/ref=cm_cr_arp_d_paging_btm_next_{}?ie=UTF8&reviewerType=all_reviews&pageNumber={}'.format(product,i,i)
                #country ='US'
            if country == '2':
                html = 'https://www.amazon.co.uk/Echo-Dot-3rd-Gen-Charcoal/product-reviews/{}/ref=cm_cr_dp_d_show_all_btm_next_{}?ie=UTF8&reviewerType=all_reviews&pageNumber={}'.format(product,i,i)
                #country ='UK'
            if country == '3':
                html = 'https://www.amazon.in/All-new-Echo-Dot-3rd-Gen/product-reviews/{}/ref=cm_cr_dp_d_show_all_btm_next_{}?ie=UTF8&reviewerType=all_reviews&pageNumber={}'.format(product,i,i)
                #country ='India'
            if country == '4':
                html = 'https://www.amazon.ca/All-new-Echo-Dot-3rd-Gen/product-reviews/{}/ref=cm_cr_dp_d_show_all_btm_next_{}?ie=UTF8&reviewerType=all_reviews&pageNumber={}'.format(product,i,i)
                #country ='Canada'
            if country == '5':
                html = 'https://www.amazon.com.au/All-new-Echo-Smart-speaker-Alexa/product-reviews/{}/ref=cm_cr_dp_d_show_all_btm_next_{}?ie=UTF8&reviewerType=all_reviews&pageNumber={}'.format(product,i,i)
                #country =' Australia'

            html = urlopen(html)
            soup = BeautifulSoup(html,"lxml")
            html = soup.prettify('utf-8')
            names = []
            full_reviews = []
            short_review = []
            long_review = []
            dates= []
            verified_pur = []

            size = soup.find('span',attrs={'class': 'a-size-base','data-hook':"cr-filter-info-review-count"}).text.strip()
            #print(size)

            for span in soup.findAll('span',attrs={'class': 'a-profile-name'}):
                name = span.text.strip()
                names.append(name)
            names =names [2::1]

            for span in soup.findAll('span',attrs={'class': ""}):
                review = span.text.strip()
                full_reviews.append(review)
                
            short_review = full_reviews[2:-5:4]
            long_review = full_reviews[3:-4:4]
            
            
            if country == '1':
                short_review = full_reviews[2::4]
                short_review.pop()
                long_review = full_reviews[3::4]
                
            if country == '2':
                short_review = full_reviews[2::4]
                long_review = full_reviews[3::4]
                        
            for span in soup.findAll('span',attrs={'class': 'a-size-base a-color-secondary review-date'}):
                date = span.text.strip()
                dates.append(date)
            dates = dates [2::1]

            for span in soup.findAll('a',attrs={'class': "a-size-mini a-link-normal a-color-secondary",'data-hook':"format-strip"}):
                verified = span.text.strip()
                verified_pur.append(verified)

            dataframe = pd.DataFrame({'Country':country,'Name':names,'Long Reviews':long_review,'Short Reviews':short_review,'Date':dates,'Product Detail':verified_pur})
            #dataframe = pd.DataFrame({'Short Reviews':short_review})
            dataframe_comp = dataframe_comp.append(dataframe,ignore_index=True)
            i = i+1

        except BaseException as ex:
            # Get current system exception
            ex_type, ex_value, ex_traceback = sys.exc_info()

            # Extract unformatter stack traces as tuples
            trace_back = traceback.extract_tb(ex_traceback)

            # Format stacktrace
            stack_trace = list()

            for trace in trace_back:
                stack_trace.append("File : %s , Line : %d, Func.Name : %s, Message : %s" % (trace[0], trace[1], trace[2], trace[3]))

                #print("Exception type : %s " % ex_type.__name__)
                #print("Exception message : %s" %ex_value)
                #print("Stack trace : %s" %stack_trace)
            e = ("Exception message : %s" %ex_value)
    
    dataframe_comp['Country'] = dataframe_comp['Country'].map({'1': 'US', '2': 'UK','3':'India','4':'Canada','5':'Australia'})
    dataframe_comp.to_csv(r'{}_Review.csv'.format(country),index=None)

US - B0792K2BK6 UK - B0792KWK57 India - B0792KTHKK Canada - B0792JYXZK Australia - B0792KRW2J